In [3]:
import warnings
warnings.filterwarnings("ignore")

# 스케일링과 변수 변환

회귀분석에 사용되는 데이터는 그 자체로 사용하기 보다는 스케일링이나 함수 변환 등의 전처리 과정을 거치는 경우가 많다. 전처리 과정은 공분산 행렬의 조건을 향상시키거나 데이터 간의 관계를 선형 모형에 맞게 바꾸기 위해 사용된다.

**조건수**(conditional number)는 공분산 행렬 $X^TX$의 가장 큰 고유치와 가장 작은 고유치의 비율을 뜻한다.

$$ \text{condition number} = \dfrac{\lambda_{\text{max}}}{\lambda_{\text{min}}} $$



In [4]:
from sklearn.datasets import load_boston
import statsmodels.api as sm
boston = load_boston()
dfX = pd.DataFrame(boston.data, columns=boston.feature_names)
dfy = pd.DataFrame(boston.target, columns=["MEDV"])
df = pd.concat([dfX, dfy], axis=1)

model1 = sm.OLS.from_formula("MEDV ~ "
                             "CRIM + ZN + INDUS + NOX + RM + AGE + "
                             "DIS + RAD + TAX + PTRATIO + B + LSTAT + CHAS", 
                             data=df)
result1 = model1.fit()
print(result1.summary())

                            OLS Regression Results                            
Dep. Variable:                   MEDV   R-squared:                       0.741
Model:                            OLS   Adj. R-squared:                  0.734
Method:                 Least Squares   F-statistic:                     108.1
Date:                Sun, 11 Mar 2018   Prob (F-statistic):          6.95e-135
Time:                        18:56:59   Log-Likelihood:                -1498.8
No. Observations:                 506   AIC:                             3026.
Df Residuals:                     492   BIC:                             3085.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     36.4911      5.104      7.149      0.0

In [5]:
result1.predict(df[:1])

0    30.008213
dtype: float64

StatsModels에서는 모형지정 문자열에서 scale 명령을 사용하여 스케일링을 할 수 있다. 

이 방식으로 스케일을 하면 스케일링에 사용된 평균과 표준편차를 저장하였다가 나중에 predict 명령을 사용할 때도 같은 스케일을 사용하기 때문에 편리하다.

In [6]:
model2 = sm.OLS.from_formula("MEDV ~ "
                             "scale(CRIM) + scale(ZN) + scale(INDUS) + scale(NOX) + scale(RM) + scale(AGE) + "
                             "scale(DIS) + scale(RAD) + scale(TAX) + scale(PTRATIO) + scale(B) + scale(LSTAT) + CHAS", 
                             data=df)
result2 = model2.fit()
print(result2.summary())

                            OLS Regression Results                            
Dep. Variable:                   MEDV   R-squared:                       0.741
Model:                            OLS   Adj. R-squared:                  0.734
Method:                 Least Squares   F-statistic:                     108.1
Date:                Sun, 11 Mar 2018   Prob (F-statistic):          6.95e-135
Time:                        18:58:34   Log-Likelihood:                -1498.8
No. Observations:                 506   AIC:                             3026.
Df Residuals:                     492   BIC:                             3085.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         22.3468      0.219    101.

## 변수 변환

다음과 같은 경우에는 로그 함수 혹은 제곱근 함수 등을 사용하여 변환된 변수를 사용하면 회귀 성능이 향상될 수도 있다.

* 독립 변수나 종속 변수가 심하게 한쪽으로 치우친 분포를 보이는 경우 
* 독립 변수와 종속 변수간의 관계가 곱셈 혹은 나눗셉으로 연결된 경우
* 종속 변수와 예측치가 비선형 관계를 보이는 경우

원래 선형회귀모형의 가정에는 독립 변수나 종속 변수가 반드시 대칭 분포를 보여야 한다는 가정은 없지만 정규 분포에 가까운 분포를 보일 수록 선형회귀모형의 성능이 좋아지는 경우가 많다.

보스턴 집값 데이터에서는 가격, DIS, PTRATIO, LSTAT 등이 비대칭 분포를 이루거나 가격과 반비례 관계를 가지므로 이 변수들을 로그로 변환한 후에 선형회귀분석을 하면 성능을 향상시킬 수 있다.

In [7]:
model3 = sm.OLS.from_formula("np.log(MEDV) ~ "
                             "scale(CRIM) + scale(ZN) + scale(INDUS) + "
                             "scale(NOX) + scale(RM) + scale(AGE) + "
                             "scale(np.log(DIS)) + scale(RAD) + scale(TAX) + "
                             "scale(np.log(PTRATIO)) + scale(B) + scale(np.log(LSTAT)) + CHAS", 
                             data=df)
result3 = model3.fit()
print(result3.summary())

                            OLS Regression Results                            
Dep. Variable:           np.log(MEDV)   R-squared:                       0.806
Model:                            OLS   Adj. R-squared:                  0.801
Method:                 Least Squares   F-statistic:                     157.4
Date:                Sun, 11 Mar 2018   Prob (F-statistic):          8.61e-166
Time:                        19:00:47   Log-Likelihood:                 150.27
No. Observations:                 506   AIC:                            -272.5
Df Residuals:                     492   BIC:                            -213.4
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                  3